## <font size=5> <strong>News Quality Prediction 
 

## I. Importing essential libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# to make this notebook's output stable across runs
np.random.seed(42)
from sklearn.metrics import confusion_matrix, f1_score
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## II. Importing and understanding our dataset 

In [29]:
dataset = pd.read_csv("https://raw.githubusercontent.com/datajour-gr/iqjournalism/main/sample_data/dataML_quality.csv")

#### Verifying it as a 'dataframe' object in pandas

In [30]:
type(dataset)

pandas.core.frame.DataFrame

In [31]:
dataset.columns

Index(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative',
       'positive', 'sadness', 'surprise', 'trust', 'valence', 'arousal',
       'dominance', 'strongsubj', ' weaksubj', 'flesch_reading_ease',
       'flesch_kincaid_grade', 'mean_word_length', 'n_stop_words', 'sources',
       'length', 'MTLD', 'Nouns', 'Verbs', 'Adjectives', 'Anger_intensity',
       'Joy_intensity', 'Sadness_intensity', 'Fear_intensity', 'No Celebs',
       'animal', 'crime', 'caps_body', 'numbers', 'urls', 'special_char_ratio',
       'quality'],
      dtype='object')

#### Shape of dataset

In [34]:
dataset.shape

(44656, 37)

In [35]:
dataset = dataset[[ 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative',
       'positive', 'sadness', 'surprise', 'trust', 'valence', 'arousal',
       'dominance', 'strongsubj', ' weaksubj',    
    'flesch_reading_ease', 'flesch_kincaid_grade', 'mean_word_length', 'n_stop_words', 
        'sources','length', 'MTLD', 'Nouns', 'Verbs', 'Adjectives', 'Anger_intensity',
       'Joy_intensity', 'Sadness_intensity', 'Fear_intensity', 'No Celebs',
       'animal', 'crime', 'caps_body', 'numbers', 'urls', 'special_char_ratio',
       'quality'] 
        ].copy()

#### Printing out a few columns

In [36]:
dataset.head(5)

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,...,Sadness_intensity,Fear_intensity,No Celebs,animal,crime,caps_body,numbers,urls,special_char_ratio,quality
0,0.980392,2.941176,0.00000,0.00000,0.980392,0.980392,0.980392,0.980392,0.00000,0.000000,...,0.059000,0.099625,0,0,0,0,2,0,2.941176,1
1,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.133000,0,0,0,0,2,0,6.060606,1
2,0.000000,0.000000,0.00000,0.00000,0.000000,1.538462,0.000000,0.000000,0.00000,3.076923,...,0.074846,0.059154,0,0,0,0,1,0,3.076923,1
3,0.000000,1.971831,0.28169,0.56338,0.563380,1.126761,3.098592,0.281690,0.28169,0.845070,...,0.098161,0.085952,0,0,0,0,0,0,2.535211,1
4,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.176429,0,0,0,0,1,0,1.562500,1


In [37]:
dataset.sample(5)

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,...,Sadness_intensity,Fear_intensity,No Celebs,animal,crime,caps_body,numbers,urls,special_char_ratio,quality
3502,0.454545,0.909091,0.000000,0.454545,0.000000,1.818182,1.363636,0.454545,0.000000,1.363636,...,0.028351,0.068541,0,0,0,0,0,0,2.272727,1
17808,0.000000,0.518135,0.000000,0.000000,0.000000,0.000000,1.554404,0.518135,0.000000,1.036269,...,0.005364,0.021250,0,0,0,1,0,0,1.554404,1
17149,1.851852,0.925926,0.925926,1.851852,0.000000,2.777778,2.777778,0.000000,0.000000,3.703704,...,0.077381,0.174857,0,0,0,0,0,0,5.555556,1
21408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,0,0,0,0,675,0,354.687500,1
21350,1.162791,2.325581,0.000000,2.325581,1.162791,3.488372,2.325581,2.325581,2.325581,2.325581,...,0.152526,0.129737,0,0,0,0,0,0,0.000000,1


#### Description

In [38]:
dataset.describe()

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,...,Sadness_intensity,Fear_intensity,No Celebs,animal,crime,caps_body,numbers,urls,special_char_ratio,quality
count,44656.000000,44656.000000,44656.000000,44656.000000,44656.000000,44656.000000,44656.000000,44656.000000,44656.000000,44656.000000,...,44470.000000,44470.000000,44656.000000,44656.000000,44656.000000,44656.000000,44656.000000,44656.000000,44656.000000,44656.000000
mean,0.817377,1.201230,0.429848,1.269040,0.812602,1.851911,2.846890,0.911296,0.530608,1.702668,...,0.091781,0.123029,0.400618,0.119939,0.405903,0.182417,5.555558,0.031530,6.705608,0.559835
std,1.094732,1.204346,0.763608,1.397372,1.030539,1.719793,1.981541,1.172217,0.799704,1.515757,...,0.074345,0.087452,0.949361,0.502454,1.273843,1.010304,45.764444,0.361004,23.566030,0.496412
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.615385,1.492537,0.000000,0.000000,0.632911,...,0.038462,0.060582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.020202,0.000000
50%,0.491602,1.020408,0.000000,0.970874,0.546448,1.515152,2.610966,0.599002,0.000000,1.449275,...,0.077625,0.107765,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,4.494382,1.000000
75%,1.265823,1.744440,0.636943,1.851852,1.250000,2.690583,3.846154,1.353014,0.843882,2.439024,...,0.128873,0.169058,1.000000,0.000000,0.000000,0.000000,3.000000,0.000000,7.695460,1.000000
max,15.662651,25.000000,13.793103,22.916667,14.141414,19.277108,22.916667,18.750000,12.500000,17.647059,...,0.738500,0.803000,24.000000,14.000000,50.000000,43.000000,2234.000000,35.000000,705.303030,1.000000


In [39]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44656 entries, 0 to 44655
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   anger                 44656 non-null  float64
 1   anticipation          44656 non-null  float64
 2   disgust               44656 non-null  float64
 3   fear                  44656 non-null  float64
 4   joy                   44656 non-null  float64
 5   negative              44656 non-null  float64
 6   positive              44656 non-null  float64
 7   sadness               44656 non-null  float64
 8   surprise              44656 non-null  float64
 9   trust                 44656 non-null  float64
 10  valence               44656 non-null  float64
 11  arousal               44656 non-null  float64
 12  dominance             44656 non-null  float64
 13  strongsubj            44656 non-null  float64
 14   weaksubj             44656 non-null  float64
 15  flesch_reading_ease

In [40]:
#WE HAVE NO MISSING VALUES

#### Let's understand our columns better:

#### Analysing the 'target' variable

In [41]:
dataset["quality"].describe()

count    44656.000000
mean         0.559835
std          0.496412
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: quality, dtype: float64

In [42]:
dataset["quality"].unique()

array([1, 0])

#### Clearly, this is a classification problem, with the target variable having values '0' and '1'

### Checking correlation between columns

In [43]:
print(dataset.corr()["quality"].abs().sort_values(ascending=False))

quality                 1.000000
 weaksubj               0.514371
flesch_kincaid_grade    0.349707
flesch_reading_ease     0.314772
Nouns                   0.305028
Joy_intensity           0.255719
Adjectives              0.207568
n_stop_words            0.180041
negative                0.179583
trust                   0.171338
fear                    0.163708
mean_word_length        0.160266
anger                   0.145196
strongsubj              0.144637
Anger_intensity         0.144499
Fear_intensity          0.127773
Sadness_intensity       0.126392
positive                0.124250
sadness                 0.124030
length                  0.105814
crime                   0.103636
Verbs                   0.099907
disgust                 0.089446
sources                 0.077749
anticipation            0.074255
MTLD                    0.065185
arousal                 0.063121
caps_body               0.060209
numbers                 0.057135
dominance               0.057009
special_ch

In [44]:
dataset = dataset.dropna()
dataset.shape

(44470, 37)

In [45]:
#This shows that most columns are moderately correlated with target.

## IV. Train Test split

In [46]:
from sklearn.model_selection import train_test_split

predictors = dataset.drop("quality",axis=1)
target = dataset["quality"]

X_train,X_test,Y_train,Y_test = train_test_split(predictors,target,test_size=0.20,random_state=0)

## V. Model Fitting

In [47]:
from sklearn.metrics import accuracy_score, classification_report

### Logistic Regression

In [48]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train,Y_train)

Y_pred_lr = lr.predict(X_test)

In [49]:
Y_pred_lr.shape

(8894,)

In [50]:
score_lr = round(accuracy_score(Y_pred_lr,Y_test)*100,2)

print("The accuracy score achieved using Logistic Regression is: "+str(score_lr)+" %")

The accuracy score achieved using Logistic Regression is: 80.77 %


In [51]:
f1_score(Y_test, Y_pred_lr, average='weighted')

0.8083275369502766

In [52]:
print('Classification Report\n', classification_report(Y_test, Y_pred_lr))

Classification Report
               precision    recall  f1-score   support

           0       0.74      0.87      0.80      3931
           1       0.88      0.76      0.82      4963

    accuracy                           0.81      8894
   macro avg       0.81      0.81      0.81      8894
weighted avg       0.82      0.81      0.81      8894



In [53]:
f1_score(Y_test, Y_pred_lr, average='weighted')

0.8083275369502766

### Naive Bayes

In [58]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_train,Y_train)

Y_pred_nb = nb.predict(X_test)

In [55]:
Y_pred_nb.shape

(8894,)

In [56]:
score_nb = round(accuracy_score(Y_pred_nb,Y_test)*100,2)

print("The accuracy score achieved using Naive Bayes is: "+str(score_nb)+" %")

The accuracy score achieved using Naive Bayes is: 80.59 %


In [57]:
f1_score(Y_test, Y_pred_nb, average='weighted')

0.8063837572315052

In [59]:
print('Classification Report\n', classification_report(Y_test, Y_pred_nb))

Classification Report
               precision    recall  f1-score   support

           0       0.73      0.88      0.80      3931
           1       0.89      0.74      0.81      4963

    accuracy                           0.81      8894
   macro avg       0.81      0.81      0.81      8894
weighted avg       0.82      0.81      0.81      8894



### SVM

In [82]:
from sklearn import svm

sv = svm.SVC(kernel='linear')

sv.fit(X_train, Y_train)

Y_pred_svm = sv.predict(X_test)

In [83]:
Y_pred_svm.shape

(8894,)

In [84]:
score_svm = round(accuracy_score(Y_pred_svm,Y_test)*100,2)

print("The accuracy score achieved using Linear SVM is: "+str(score_svm)+" %")

The accuracy score achieved using Linear SVM is: 81.8 %


In [85]:
f1_score(Y_test, Y_pred_svm, average='weighted')

0.8179552371216816

In [86]:
print('Classification Report\n', classification_report(Y_test, Y_pred_svm))

Classification Report
               precision    recall  f1-score   support

           0       0.73      0.93      0.82      3931
           1       0.93      0.73      0.82      4963

    accuracy                           0.82      8894
   macro avg       0.83      0.83      0.82      8894
weighted avg       0.84      0.82      0.82      8894



### K Nearest Neighbors

In [60]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train,Y_train)
Y_pred_knn=knn.predict(X_test)

In [61]:
Y_pred_knn.shape

(8894,)

In [62]:
score_knn = round(accuracy_score(Y_pred_knn,Y_test)*100,2)

print("The accuracy score achieved using KNN is: "+str(score_knn)+" %")

The accuracy score achieved using KNN is: 80.2 %


In [63]:
f1_score(Y_test, Y_pred_knn, average='weighted')

0.8025505539336572

In [64]:
print('Classification Report\n', classification_report(Y_test, Y_pred_knn))

Classification Report
               precision    recall  f1-score   support

           0       0.73      0.87      0.80      3931
           1       0.88      0.75      0.81      4963

    accuracy                           0.80      8894
   macro avg       0.81      0.81      0.80      8894
weighted avg       0.81      0.80      0.80      8894



### Decision Tree

In [72]:
from sklearn.tree import DecisionTreeClassifier

max_accuracy = 0

for x in range(100):
    dt = DecisionTreeClassifier(random_state=x, max_depth = 8)
    dt.fit(X_train,Y_train)
    Y_pred_dt = dt.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_dt,Y_test)*100,2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x
        
#print(max_accuracy)
#print(best_x)

dt = DecisionTreeClassifier(random_state=best_x)
dt.fit(X_train,Y_train)
Y_pred_dt = dt.predict(X_test)

In [73]:
print(Y_pred_dt.shape)

(8894,)


In [74]:
score_dt = round(accuracy_score(Y_pred_dt,Y_test)*100,2)

print("The accuracy score achieved using Decision Tree is: "+str(score_dt)+" %")

The accuracy score achieved using Decision Tree is: 79.28 %


In [75]:
f1_score(Y_test, Y_pred_dt, average='weighted')

0.7927789075073154

In [76]:
print('Classification Report\n', classification_report(Y_test, Y_pred_dt))

Classification Report
               precision    recall  f1-score   support

           0       0.77      0.77      0.77      3931
           1       0.81      0.81      0.81      4963

    accuracy                           0.79      8894
   macro avg       0.79      0.79      0.79      8894
weighted avg       0.79      0.79      0.79      8894



### Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier

max_accuracy = 0


for x in range(2):
    rf = RandomForestClassifier(random_state=x, min_samples_split=3, n_estimators=500,max_features=0.9)
    rf.fit(X_train,Y_train)
    Y_pred_rf = rf.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_rf,Y_test)*100,2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x
        
#print(max_accuracy)
#print(best_x)

rf = RandomForestClassifier(random_state=best_x)
rf.fit(X_train,Y_train)
Y_pred_rf = rf.predict(X_test)

In [20]:
Y_pred_rf.shape

(8894,)

In [21]:
score_rf = round(accuracy_score(Y_pred_rf,Y_test)*100,2)

print("The accuracy score achieved using Decision Tree is: "+str(score_rf)+" %")

The accuracy score achieved using Decision Tree is: 84.87 %


In [22]:
f1_score(Y_test, Y_pred_rf, average='weighted')

0.84905410083448

In [23]:
print('Classification Report\n', classification_report(Y_test, Y_pred_rf))

Classification Report
               precision    recall  f1-score   support

           0       0.78      0.93      0.84      3931
           1       0.93      0.79      0.85      4963

    accuracy                           0.85      8894
   macro avg       0.85      0.86      0.85      8894
weighted avg       0.86      0.85      0.85      8894



In [24]:
#2
rf = RandomForestClassifier(n_estimators=2000, min_samples_leaf=4, max_features=0.5, n_jobs=-1, oob_score=True)
rf.fit(X_train,Y_train)

RandomForestClassifier(max_features=0.5, min_samples_leaf=4, n_estimators=2000,
                       n_jobs=-1, oob_score=True)

In [25]:
f1_score(Y_test, Y_pred_rf, average='weighted')

0.84905410083448

In [27]:
n_nodes = []
max_depths = []

for ind_tree in rf.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Total number of Trees {int(len(rf.estimators_))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')


Total number of Trees 2000
Average number of nodes 3124
Average maximum depth 31


In [ ]:
# visualizing the nth tree of the Random Forest (n=0)
import graphviz
from sklearn.tree import export_graphviz

#dot_data = export_graphviz(rf.estimators_[0])
graphviz.Source(export_graphviz(rf.estimators_[10],
                                out_file=None,
                                feature_names=features,
                                class_names=["0", "1"],
                                node_ids = True,
                                impurity=True))

### XGBoost

In [66]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="auc")
xgb_model.fit(X_train, Y_train, early_stopping_rounds=5, eval_set=[(X_test, Y_test)])

Y_pred_xgb = xgb_model.predict(X_test)

[0]	validation_0-auc:0.90926
[1]	validation_0-auc:0.91572
[2]	validation_0-auc:0.91745
[3]	validation_0-auc:0.91990
[4]	validation_0-auc:0.92055
[5]	validation_0-auc:0.92172
[6]	validation_0-auc:0.92275
[7]	validation_0-auc:0.92289
[8]	validation_0-auc:0.92433
[9]	validation_0-auc:0.92479
[10]	validation_0-auc:0.92575
[11]	validation_0-auc:0.92626
[12]	validation_0-auc:0.92658
[13]	validation_0-auc:0.92747
[14]	validation_0-auc:0.92742
[15]	validation_0-auc:0.92795
[16]	validation_0-auc:0.92820
[17]	validation_0-auc:0.92865
[18]	validation_0-auc:0.92873
[19]	validation_0-auc:0.92867
[20]	validation_0-auc:0.92891
[21]	validation_0-auc:0.92923
[22]	validation_0-auc:0.92943
[23]	validation_0-auc:0.92948
[24]	validation_0-auc:0.92970
[25]	validation_0-auc:0.92970
[26]	validation_0-auc:0.92984
[27]	validation_0-auc:0.92958
[28]	validation_0-auc:0.92959
[29]	validation_0-auc:0.92965
[30]	validation_0-auc:0.92912
[31]	validation_0-auc:0.92895


In [67]:
print("best score: {0}, best iteration: {1}".format(xgb_model.best_score, xgb_model.best_iteration))

best score: 0.9298375262621342, best iteration: 26


In [68]:
f1_score(Y_test, Y_pred_xgb, average='weighted')

0.845576658765714

In [69]:
score_xgb = round(accuracy_score(Y_pred_xgb,Y_test)*100,2)

print("The accuracy score achieved using XGBoost is: "+str(score_xgb)+" %")

The accuracy score achieved using XGBoost is: 84.52 %


In [70]:
print('Classification Report\n', classification_report(Y_test, Y_pred_xgb))

Classification Report
               precision    recall  f1-score   support

           0       0.77      0.92      0.84      3931
           1       0.93      0.78      0.85      4963

    accuracy                           0.85      8894
   macro avg       0.85      0.85      0.85      8894
weighted avg       0.86      0.85      0.85      8894

